In [1]:
import pandas as pd
import numpy as np
import os
from agents import Runner, agent, trace, set_trace_processors, Agent, ModelSettings
from weave.integrations.openai_agents.openai_agents import WeaveTracingProcessor
import asyncio
from typing import Literal, List, Annotated
import weave
from dotenv import load_dotenv
from pydantic import BaseModel, Field, conlist
from IPython.display import display, Markdown, HTML, Image
from tqdm import tqdm
import warnings
load_dotenv()
# Import Necessary Libraries
from src.Agents.Master_Agent import q_a_agent
# Import Weave and Biases
weave.init("Operation Model")
set_trace_processors([WeaveTracingProcessor()])
warnings.filterwarnings("ignore")

weave: Logged in as Weights & Biases user: francisco-reveriano-1.
weave: View Weave data at https://wandb.ai/francisco-reveriano-1-mckinsey-company/operation-model/weave


## Single Query

In [ ]:
with trace("Test Query"):
    message = "What is the 5-year cumulative percentage change that Generative AI will have on the following process: Quality Improvement & Reporting within the following Line of Business (LoB): Mortgage Servicing? Keep in mind that the bank's current onshore capacity is 60.0 Full-Time Equivalent Employees."
    response = await Runner.run(q_a_agent, message, max_turns=100)

print(response.final_output)

In [ ]:
response.final_output.use_online_coursework

## Load Dataset

In [ ]:
excel_sheet = pd.ExcelFile("../Data/Raw/Ops_Overview_Data_File.xlsx")
# List all sheet names
print(excel_sheet.sheet_names)

In [ ]:
df = None
df = pd.read_excel("../Data/Raw/Ops_Overview_Data_File.xlsx", sheet_name="Mortgage Servicing")
df.fillna(0, inplace=True)
df["ONSHORE"] = df[['ONSHORE TEAMMATE', 'ONSHORE CW', 'Est. Size- ONSHORE ']].sum(axis=1)
df["OFFSHORE"] = df[["Est. Size- OFFSHORE"]]
df.head(2)

In [ ]:
import asyncio

sheet_name = "Mortgage Servicing"

async def run_queries(row):
    process   = row["Function / Process Name"]
    onshore   = row["ONSHORE"]
    offshore  = row["OFFSHORE"]

     # Build the two prompts
    onshore_query = (
        f"What is the 5-year cumulative percentage change that Generative AI will have "
        f"on the following process: {process} within the following Line of Business (LoB): "
        f"{sheet_name}? Keep in mind that the bank's current onshore capacity is {onshore}"
    )
    offshore_query = (
        f"What is the 5-year cumulative percentage change that Generative AI will have "
        f"on the following process: {process} within the following Line of Business (LoB): "
        f"{sheet_name}? Keep in mind that the bank's current offshore capacity is {offshore}"
    )
    # Fire both queries concurrently
    with trace(sheet_name):
        on_task  = asyncio.create_task(Runner.run(q_a_agent, onshore_query, max_turns=100))
        off_task = asyncio.create_task(Runner.run(q_a_agent, offshore_query, max_turns=100))
    on, off  = await asyncio.gather(on_task, off_task)

    return {
        # ---------- On-shore ----------
        "Onshore_High_Scenario_Vectors":    on.final_output.high_scenario,
        "Onshore_Medium_Scenario_Vectors":  on.final_output.medium_scenario,
        "Onshore_Low_Scenario_Vectors":     on.final_output.low_scenario,
        "Onshore_High_Scenario_Reasoning":  on.final_output.high_scenario_reasoning,
        "Onshore_Medium_Scenario_Reasoning":on.final_output.medium_scenario_reasoning,
        "Onshore_Low_Scenario_Reasoning":   on.final_output.low_scenario_reasoning,
        "Onshore_CourseWork":               on.final_output.online_coursework,
        # ---------- Off-shore ----------
        "Offshore_High_Scenario_Vectors":    off.final_output.high_scenario,
        "Offshore_Medium_Scenario_Vectors":  off.final_output.medium_scenario,
        "Offshore_Low_Scenario_Vectors":     off.final_output.low_scenario,
        "Offshore_High_Scenario_Reasoning":  off.final_output.high_scenario_reasoning,
        "Offshore_Medium_Scenario_Reasoning":off.final_output.medium_scenario_reasoning,
        "Offshore_Low_Scenario_Reasoning":   off.final_output.low_scenario_reasoning,
        "Offshore_CourseWork":               off.final_output.online_coursework,
    }

async def main():
    coros = [run_queries(row) for _, row in df.iterrows()]
    results = await asyncio.gather(*coros)                       #  ◀️ massive fan-out
    df_out = pd.concat([df, pd.DataFrame(results)], axis=1)
    return df_out

df = await main()
df.to_excel("../Data/Intermediate/Ops_Overview_Data_File_Processed.xlsx", sheet_name=sheet_name, index=False)# ✔️ Jupyter’s loop drives it
df.head()
